In [1]:
# Import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [2]:
# Import webdriver for webscraping U.S. congress website
website = 'https://www.congress.gov'
driver = webdriver.Chrome()
driver.get(website)

In [3]:
# Select Hong Kong or Hongkonger or Hongkongers as keywords in the search
text_box = driver.find_element(By.XPATH, '//input[@class="locsuggest"]')
text_box.send_keys("Hong Kong or Hongkonger or Hongkongers")
submit_button = driver.find_element(By.XPATH, '//button[@id="search-submit"]')
submit_button.click()

In [4]:
# Uncheck all options for "Limit Your Search" 
time.sleep(10)
uncheck_1 = driver.find_element(By.XPATH, '//a[@id="checkAllsource"]')
uncheck_1.click()

In [5]:
# Check "Congressional Record" option
time.sleep(10)
record_box1 = driver.find_element(By.XPATH, '//input[@id="facetItemsourceCongressional_Recordinput"]')
driver.execute_script("arguments[0].click();", record_box1)

In [6]:
# Check "Committee Meetings" option
time.sleep(10)
record_box2 = driver.find_element(By.XPATH, '//input[@id="facetItemsourceCommittee_Meetingsinput"]')
driver.execute_script("arguments[0].click();", record_box2)

In [7]:
# Check "116 (2019-2020)" option for "Congress"
time.sleep(10)
congress_box1 = driver.find_element(By.XPATH, '//input[@id="facetItemcongress116__2019-2020_input"]')
driver.execute_script("arguments[0].click();", congress_box1)

In [8]:
# Uncheck "119 (2025-2026)" option
time.sleep(10)
congress_box2 = driver.find_element(By.XPATH, '//input[@id="facetItemcongress119__2025-2026_input"]')
driver.execute_script("arguments[0].click();", congress_box2)

In [9]:
# Uncheck "118 (2023-2024)" option
time.sleep(10)
congress_box3 = driver.find_element(By.XPATH, '//input[@id="facetItemcongress118__2023-2024_input"]')
driver.execute_script("arguments[0].click();", congress_box3)

In [10]:
# Select "Latest Action - Oldest to Newest" option from the dropdown box
time.sleep(10)
dropdown = Select(driver.find_element(By.XPATH, '//select[@id="pageSort"]'))
dropdown.select_by_visible_text('Latest Action - Oldest to Newest')

In [11]:
# Locate the pages buttons
pagination = driver.find_elements(By.XPATH, '//div[@class="pagination"]')[1]
pages = pagination.find_elements(By.TAG_NAME, 'a')

In [12]:
current_page = 1
last_page = int(pages[3].text) + 1
href_list = []

In [13]:
# Obtain a list of hyperlinks for relevant congressional records and committee meetings
while current_page <= last_page:
    time.sleep(10)
    box = driver.find_element(By.XPATH, '//div[@id="main"]')
    meetings = box.find_elements(By.XPATH, '//span[@class="result-heading congressional-record-heading" or @class="result-heading"]')
    for meeting in meetings:
        href_list.append(meeting.find_element(By.TAG_NAME, 'a').get_attribute('href'))

    current_page = current_page + 1

    try:
        next_page = driver.find_element(By.XPATH, '//a[@class="next"]')
        next_page.click()
    except:
        pass

driver.quit()

In [14]:
# Transform to a unique list of hyperlinks
def get_unique_elements(input_list):
    seen = set()
    unique_list = []
    for item in input_list:
        if item not in seen:
            unique_list.append(item)
            seen.add(item)
    return unique_list

unique_list = get_unique_elements(href_list)

In [15]:
# There are 501 hyperlinks in the unique list
print(len(unique_list))

300


In [ ]:
# Extract txt files for each transcript
number = 0
for link in unique_list:
    time.sleep(3)
    driver = webdriver.Chrome()
    driver.get(link)
    number = number + 1
    transcript = driver.find_element(By.TAG_NAME, 'pre').text
    with open(f"transcript_{number}.txt", "w") as file:
        file.write(transcript)
    driver.quit()